In [237]:
import re
import requests
import cookielib
from bs4 import BeautifulSoup


session = requests.session()
r = session.get('http://www.eomf.ou.edu/photos/clusters.kml?bbox=-272.58203125,-110.9825683914,270.58203124999,112.1074894953')

soup = BeautifulSoup(r.text, 'lxml')

In [238]:
clusters = []
for cluster in soup.find_all('placemark'):
    tmp_dict = {}
    for data in cluster.extendeddata.find_all('data'):
        key = data['name']
        value = float(data.value.string) if '.' in data.value.string else int(data.value.string)
        tmp_dict[key] = value
    clusters.append(tmp_dict)


In [ ]:
from math import ceil
for c in clusters:
    for index in range(1, int(ceil(c['count'] / 192.0)) + 1):
        r = session.get('http://www.eomf.ou.edu/photos/photos.html?ids={}&x_size={}&y_size={}&page={}&ppp=192'\
                        .format(c['ids'], c['x_size'], c['y_size'], index))
        print r, r.url
        if r.status_code != 200:
            continue
        soup = BeautifulSoup(r.text, 'lxml')
        body = soup.html.body.contents[2]
        ids = []
        token = body.find("input", {"name":"csrfmiddlewaretoken"})["value"]
        for pic in body.find_all("div", class_="photo-tile"):
            parameters = {"ids": pic.input['id'], "selector": "Off", "format": "csv", "img": "big", "csrfmiddlewaretoken": token}
            print parameters["ids"]
            headers = {'Content-Type': 'application/x-www-form-urlencoded'}
            r = session.post("http://www.eomf.ou.edu/photos/download/", data = parameters, headers=headers, stream=True)
            if r.status_code != 200:
                continue
            z = zipfile.ZipFile(StringIO.StringIO(r.content))
            z.extractall("E://EOMF//"+ pic.input['id'] + "//")
        

<Response [200]> http://www.eomf.ou.edu/photos/photos.html?ids=8596&x_size=10.863281&y_size=7.242187&page=1&ppp=192
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611

In [222]:
with open("F://1.zip", 'wb') as f:
    f.write(r.conte)
# import zipfile, StringIO
# z = zipfile.ZipFile(StringIO.StringIO(r.content))
# z.extractall("F://1//")